In [1]:
#!/usr/bin/env python
#This script is used to generate training and testing datasets.
import click as ck
import numpy as np
import pandas as pd
from collections import Counter
from utils import Ontology, FUNC_DICT
import logging

In [2]:
def main(go_file, hp_file, hp_annots_file, deepgo_annots_file, id_mapping_file,
         data_file, string_mapping_file, expressions_file,
         out_data_file, out_terms_file, min_count):
    go = Ontology(go_file, with_rels=True)
    print('GO loaded')
    hp = Ontology(hp_file, with_rels=True)
    print('HP loaded')
    print('Load Gene2prot mapping')
    df = pd.read_pickle(data_file)
    print('length of swissprot=',len(df))
    prot2gene = {}
    with open(id_mapping_file) as f:
        next(f)
        for line in f:
            it = line.strip().split('\t')
            prot2gene[it[2]] = it[0]
    # for row in df.itertuples():
    #     if row.genes != '':
    #         prot2gene[row.proteins] = row.genes
    
    print('Loading HP annotations')
    hp_annots = {}
    name2gene = {}
    with open(hp_annots_file) as f:
        next(f)
        for line in f:
            it = line.strip().split('\t')
            gene_id = it[0]
            gene_name = it[1].upper()
            name2gene[gene_name] = gene_id
            hp_id = it[3]
            if gene_id not in hp_annots:
                hp_annots[gene_id] = set()
            if hp.has_term(hp_id):
                hp_annots[gene_id] |= hp.get_anchestors(hp_id)
    total_annots = 0
    for g_id, annots in hp_annots.items():
        total_annots += len(annots)
    print('HP Annotations', len(hp_annots), total_annots, (total_annots / len(hp_annots)))
    dg_annots = {}
    gos = set()
    
    
    
#     with open(deepgo_annots_file) as f:
#         for line in f:
#             it = line.strip().split('\t')
#             if it[0] not in prot2gene:
#                 continue
#             gene_id = prot2gene[it[0]]
#             annots = dg_annots.get(gene_id, {})
#             for item in it[1:]:
#                 go_id, score = item.split('|')
#                 score = float(score)
#                 annots[go_id] = max(score, annots.get(go_id, 0))
#             dg_annots[gene_id] = annots
#             gos |= set(annots.keys())
#     print('DeepGO Annotations', len(dg_annots))
#     deepgo_annots = {}
#     for g_id, annots in dg_annots.items():
#         deepgo_annots[g_id] = [go_id + '|' + str(score) for go_id, score in annots.items()]
#     print('Number of GOs', len(gos))


    gos_df = pd.DataFrame({'gos': list(gos)})
    gos_df.to_pickle('data/My_Implementations/Trial_7/gos.pkl')

    go_annots = {}
    iea_annots = {}
    seqs = {}
    print('lentgth of prot2gene',len(prot2gene))
    for i, row in df.iterrows():
        if row.proteins not in prot2gene:
            continue
        g_id = prot2gene[row.proteins]
        if g_id not in go_annots:
            go_annots[g_id] = set()
            iea_annots[g_id] = set()
        go_annots[g_id] |= set(row.exp_annotations)
        iea_annots[g_id] |= set(row.iea_annotations)
        seqs[g_id] = row.sequences

    print('GO Annotations', len(go_annots))
    logging.info('Processing annotations')
    
    cnt = Counter()
    annotations = list()
    for g_id, annots in hp_annots.items():
        for term in annots:
            cnt[term] += 1

    gene_exp = {}
    max_val = 0
    with open(expressions_file) as f:
        for line in f:
            if line.startswith('#') or line.startswith('Gene'):
                continue
            it = line.strip().split('\t')
            gene_name = it[1].upper()
            if gene_name in name2gene:
                exp = np.zeros((53,), dtype=np.float32)
                for i in range(len(it[2:])):
                    exp[i] = float(it[2 + i]) if it[2 + i] != '' else 0.0
                gene_exp[name2gene[gene_name]] = exp / np.max(exp)
                
    print('Expression values', len(gene_exp))
    counter=0
    deepgo_annotations = []
    go_annotations = []
    iea_annotations = []
    hpos = []
    genes = []
    sequences = []
    expressions = []
    mis_exp = 0
    c=415
#     print(go_annots)
#     for x in go_annots:
#         print (x)
    if c not in go_annots:
        print("Yes")

    for g_id, phenos in hp_annots.items():
        if g_id not in go_annots:
            continue
        else:
            genes.append(g_id)
            hpos.append(phenos)
            go_annotations.append(go_annots[g_id])
            iea_annotations.append(iea_annots[g_id])
#             deepgo_annotations.append(deepgo_annots[g_id])
            sequences.append(seqs[g_id])
            if g_id in gene_exp:
                expressions.append(gene_exp[g_id])
            else:
                expressions.append(np.zeros((53,), dtype=np.float32))
                mis_exp += 1
    print('Missing expressions', mis_exp)
    
#     for g_id, gos in dg_annots.items():
#         genes.append(g_id)
#         phenos = set()
#         if g_id in hp_annots:
#             phenos = hp_annots[g_id]
#         hpos.append(phenos)
#         go_annotations.append(go_annots[g_id])
#         iea_annotations.append(iea_annots[g_id])
#         deepgo_annotations.append(deepgo_annots[g_id])
#         sequences.append(seqs[g_id])
    
        
    df = pd.DataFrame(
        {'genes': genes, 'hp_annotations': hpos,
         'go_annotations': go_annotations, 'iea_annotations': iea_annotations,
#          'deepgo_annotations': deepgo_annotations,
         'sequences': sequences, 'expressions': expressions})
    df.to_pickle(out_data_file)
    print(f'Number of proteins {len(df)}')
    print(df)
    # Filter terms with annotations more than min_count
    terms_set = set()
    all_terms = []
    for key, val in cnt.items():
        if key == 'HP:0000001':
            continue
        all_terms.append(key)
        if val >= min_count:
            terms_set.add(key)
    terms = []
    labels = []
    for t_id in hp.get_ordered_terms():
        if t_id in terms_set:
            terms.append(t_id)
            labels.append(hp.get_term(t_id)['name'])
    
    logging.info(f'Number of terms {len(terms)}')
    # Save the list of terms
    df = pd.DataFrame({'terms': terms, 'labels': labels})
    df.to_pickle(out_terms_file)
    df = pd.DataFrame({'terms': all_terms})
#     df.to_pickle('data/My_Implementations/Trial_7/all_terms.pkl')
    df.to_pickle('data/My_Implementations/Trial_7/mouse_all_terms.pkl')

In [18]:
#go_file,

#hp_file, 

#hp_annots_file, 

#deepgo_annots_file, 

#id_mapping_file,

#data_file, string_mapping_file,

#out_data_file, 

#out_terms_file, 

#min_count):

main('data/go.obo',
     
     'data/hp.obo',
     
     'data/ALL_SOURCES_ALL_FREQUENCIES_genes_to_phenotype.txt',
     
     'data/human.res',
     
     'data/My_Implementations/gene2prot.tab',
     
     'data/My_Implementations/swissprot_version.pkl',
     
     'data/string2uni.tab',
     
     'data/My_Implementations/E-MTAB-5214-query-results.tpms.tsv',
     
     'data/My_Implementations/Trial_7/human_trail_7.pkl',
     
     'data/My_Implementations/Trial_7/terms_trail_7.pkl',
     
     1)

GO loaded
HP loaded
Load Gene2prot mapping
length of swissprot= 20365
Loading HP annotations
HP Annotations 4073 533548 130.99631721090105
lentgth of prot2gene 3976
GO Annotations 3939
Expression values 3980
Yes
Missing expressions 53
Number of proteins 3933
       genes                                     hp_annotations  \
0       8192  {HP:0000008, HP:0011389, HP:0008527, HP:000868...   
1          2               {HP:0000005, HP:0000006, HP:0000001}   
2       8195  {HP:0012372, HP:0000036, HP:0000175, HP:000046...   
3       8200  {HP:0006495, HP:0009237, HP:0010580, HP:001138...   
4      90121  {HP:0000175, HP:0001909, HP:0009122, HP:001093...   
5       8204  {HP:0010946, HP:0003829, HP:0012210, HP:000001...   
6         16  {HP:0100022, HP:0012372, HP:0011389, HP:003182...   
7         18  {HP:0100022, HP:0011420, HP:0031826, HP:000437...   
8         19  {HP:0100022, HP:0011004, HP:0011492, HP:001237...   
9         21  {HP:0031653, HP:0003674, HP:0100759, HP:002542...   
10  

## Creating Data for the new release

In [3]:
#go_file,

#hp_file, 

#hp_annots_file, 

#deepgo_annots_file, 

#id_mapping_file,

#data_file, string_mapping_file,

#out_data_file, 

#out_terms_file, 

#min_count):

main('data/go.obo',
     
     'data/hp_10_2021.obo',
     
     'data/genes_to_phenotype_10_2021.txt',
     
     'data/human.res',
     
     'data/My_Implementations/gene2prot.tab',
     
     'data/My_Implementations/swissprot_version.pkl',
     
     'data/string2uni.tab',
     
     'data/My_Implementations/E-MTAB-5214-query-results.tpms.tsv',
     
     'data/My_Implementations/Trial_7/human_10_2021.pkl',
     
     'data/My_Implementations/Trial_7/terms_10_2021.pkl',
     
     1)

GO loaded
HP loaded
Load Gene2prot mapping
length of swissprot= 20365
Loading HP annotations
HP Annotations 4707 0 0.0
lentgth of prot2gene 3976
GO Annotations 3939
Expression values 4603
Yes
Missing expressions 56
Number of proteins 3921
       genes hp_annotations  \
0       8192             {}   
1       8195             {}   
2       8200             {}   
3      90121             {}   
4       8204             {}   
...      ...            ...   
3916  147409             {}   
3917    8148             {}   
3918  344018             {}   
3919   81887             {}   
3920   57338             {}   

                                         go_annotations  \
0     {GO:0043227, GO:0032991, GO:0005737, GO:000562...   
1     {GO:0005737, GO:0030705, GO:0051640, GO:000681...   
2     {GO:0071495, GO:0006357, GO:0051241, GO:004885...   
3                                                    {}   
4     {GO:0044428, GO:0071495, GO:0051254, GO:000635...   
...                               

In [4]:
human_df = pd.read_pickle('data/My_Implementations/Trial_7/human_10_2021.pkl')
human_df

,genes,hp_annotations,go_annotations,iea_annotations,sequences,expressions
0,8192,{},"{GO:0043227, GO:0032991, GO:0005737, GO:000998...","{GO:0043227, GO:0032991, GO:0005737, GO:004423...",MWPGILVGGARVASCRYPALGPRLAAHFPAQRPPQRTLQNGLALQR...,"[0.19444445, 0.21296297, 0.23148148, 1.0, 0.46..."
1,8195,{},"{GO:0051640, GO:0006810, GO:0048856, GO:006027...","{GO:0007606, GO:0060632, GO:0006810, GO:004001...",MSRLEAKKPSLCKSEPLTTERVRTTLSVLKRIVTSCYGPSGRLKQL...,"[0.5, 0.71428573, 0.39285713, 0.4642857, 0.428..."
2,8200,{},"{GO:0071495, GO:0006357, GO:0051241, GO:004885...","{GO:0043069, GO:0007275, GO:0070997, GO:007149...",MRLPKLLTFLLWYLAWLDLEFICTVLGAPDLGQRPQGTRPGLAKAE...,"[0.0125, 0.025, 0.025, 0.025, 0.025, 0.0125, 0..."
3,90121,{},{},"{GO:0044237, GO:0009987, GO:0044238, GO:004225...",MAGAAEDARALFRAGVCAALEAWPALQIAVENGFGGVHSQEKAKWL...,"[0.62211984, 0.76958525, 1.0, 0.41013825, 0.36..."
4,8204,{},"{GO:0044428, GO:0071495, GO:0051254, GO:000635...","{GO:0007275, GO:0044428, GO:0071495, GO:005125...",MTHGEELGSDVHQDSIVLTYLEGLLMHQAAGGSGTAVDKKSAGHNE...,"[0.10714286, 0.17857143, 0.17857143, 0.25, 0.4..."
...,...,...,...,...,...,...
3916,147409,{},"{GO:0007275, GO:0009987, GO:0008150, GO:000382...","{GO:0071495, GO:0006357, GO:0048856, GO:004424...",MDWLFFRNICLLIILMVVMEVNSEFIVEVKEFDIENGTTKWQTVRR...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3917,8148,{},"{GO:0044428, GO:0043489, GO:0051254, GO:009715...","{GO:0044428, GO:0043489, GO:0051254, GO:009715...",MSDSGSYGQSGGEQQSYSTYGNPGSQGYGQASQSYSGYGQTTDSSY...,"[0.2644628, 0.32231405, 0.446281, 1.0, 0.44628..."
3918,344018,{},"{GO:0044424, GO:0032991, GO:0003674, GO:000562...","{GO:0007275, GO:0044428, GO:0009994, GO:005125...",MDPAPGVLDPRAAPPALLGTPQAEVLEDVLREQFGPLPQLAAVCRL...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3919,81887,{},"{GO:0043227, GO:0031981, GO:0044428, GO:003299...","{GO:0043227, GO:0044428, GO:0031981, GO:004444...",MSWESGAGPGLGSQGMDLVWSAWYGKCVKGKGSLPLSAHGIVVAWL...,"[0.18965517, 0.23275863, 0.29310346, 0.3620689..."


In [ ]:
human_df

In [19]:
human_df = pd.read_pickle('data/My_Implementations/Trial_7/human_trail_7.pkl')
human_df


,genes,hp_annotations,go_annotations,iea_annotations,sequences,expressions
0,8192,"{HP:0000008, HP:0011389, HP:0008527, HP:000868...","{GO:0009368, GO:0030163, GO:0009056, GO:000815...","{GO:0009368, GO:0030163, GO:0009056, GO:000815...",MWPGILVGGARVASCRYPALGPRLAAHFPAQRPPQRTLQNGLALQR...,"[0.19444445, 0.21296297, 0.23148148, 1.0, 0.46..."
1,2,"{HP:0000005, HP:0000006, HP:0000001}","{GO:0007596, GO:0032102, GO:0044260, GO:005254...","{GO:0030154, GO:0007596, GO:0032102, GO:004426...",MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,"[0.015174977, 0.014245897, 0.03995045, 0.00061..."
2,8195,"{HP:0012372, HP:0000036, HP:0000175, HP:000046...","{GO:0042995, GO:0008104, GO:0061629, GO:004444...","{GO:0042995, GO:0030154, GO:0010467, GO:003210...",MSRLEAKKPSLCKSEPLTTERVRTTLSVLKRIVTSCYGPSGRLKQL...,"[0.5, 0.71428573, 0.39285713, 0.4642857, 0.428..."
3,8200,"{HP:0009237, HP:0010580, HP:0011420, HP:000953...","{GO:0030154, GO:0009719, GO:0010467, GO:007088...","{GO:0030154, GO:0009719, GO:0010467, GO:000679...",MRLPKLLTFLLWYLAWLDLEFICTVLGAPDLGQRPQGTRPGLAKAE...,"[0.0125, 0.025, 0.025, 0.025, 0.025, 0.0125, 0..."
4,90121,"{HP:0000175, HP:0010938, HP:0008373, HP:020000...",{},"{GO:0000462, GO:0010467, GO:0008150, GO:007184...",MAGAAEDARALFRAGVCAALEAWPALQIAVENGFGGVHSQEKAKWL...,"[0.62211984, 0.76958525, 1.0, 0.41013825, 0.36..."
5,8204,"{HP:0003829, HP:0012210, HP:0000014, HP:000000...","{GO:0009719, GO:0003677, GO:0010467, GO:007088...","{GO:0009719, GO:0003677, GO:0010467, GO:007088...",MTHGEELGSDVHQDSIVLTYLEGLLMHQAAGGSGTAVDKKSAGHNE...,"[0.10714286, 0.17857143, 0.17857143, 0.25, 0.4..."
6,16,"{HP:0100022, HP:0012372, HP:0011389, HP:003182...","{GO:0010467, GO:0008150, GO:0140098, GO:004426...","{GO:0010467, GO:0044260, GO:0006417, GO:000557...",MDSTLTASEIRQRFIDFFKRNEHTYVHSSATIPLDDPTLLFANAGM...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,18,"{HP:0100022, HP:0011420, HP:0031826, HP:000682...","{GO:0001505, GO:0008144, GO:0008150, GO:000652...","{GO:0042995, GO:0007596, GO:0032102, GO:000961...",MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLM...,"[0.8369565, 0.8913044, 0.42391303, 0.02173913,..."
8,19,"{HP:0100022, HP:0011492, HP:0012372, HP:000171...","{GO:0055092, GO:0030154, GO:0009719, GO:000690...","{GO:0055092, GO:0030154, GO:0036314, GO:003437...",MACWPQLRLLLWKNLTFRRRQTCQLLLEVAWPLFIFLILISVRLSY...,"[0.046875, 0.03125, 0.0625, 0.00625, 1.0, 0.07..."
9,21,"{HP:0031653, HP:0003674, HP:0100759, HP:002542...","{GO:0050896, GO:0042221, GO:0071944, GO:000815...","{GO:0009719, GO:0044260, GO:0042599, GO:000557...",MAVLRQLALLLWKNYTLQKRKVLVTVLELFLPLLFSGILIWLRLKI...,"[0.21192053, 0.25827813, 0.17880794, 0.0198675..."


In [4]:
#go_file,

#hp_file, 

#hp_annots_file, 

#deepgo_annots_file, 

#id_mapping_file,

#data_file, string_mapping_file,

#out_data_file, 

#out_terms_file, 

#min_count):

main('data/go.obo',
     
     'data/hp.obo',
     
     'data/ALL_SOURCES_ALL_FREQUENCIES_genes_to_phenotype.txt',
     
     'data/human.res',
     
     'data/My_Implementations/gene2prot.tab',
     
     'data/My_Implementations/swissprot_version_mouse.pkl',
     
     'data/string2uni.tab',
     
     'data/My_Implementations/E-MTAB-5214-query-results.tpms.tsv',
     
     'data/My_Implementations/Trial_7/mouse_trail_7.pkl',
     
     'data/My_Implementations/Trial_7/mouse_terms_trail_7.pkl',
     
     1)

GO loaded
HP loaded
Load Gene2prot mapping
length of swissprot= 17033
Loading HP annotations
HP Annotations 4073 533548 130.99631721090105
lentgth of prot2gene 3976
GO Annotations 0
Expression values 3980
Yes
Missing expressions 0
Number of proteins 0
Empty DataFrame
Columns: [genes, hp_annotations, go_annotations, iea_annotations, sequences, expressions]
Index: []


In [34]:
human_df = pd.read_pickle('data/My_Implementations/human.pkl')

In [35]:
human_df

,genes,hp_annotations,go_annotations,iea_annotations,deepgo_annotations,sequences,expressions
0,8192,"{HP:0000008, HP:0011389, HP:0001250, HP:000059...","{GO:0003674, GO:0043227, GO:0008150, GO:003361...","{GO:0042802, GO:0003674, GO:0043227, GO:000551...","[GO:0000502|0.052, GO:0001539|0.035, GO:000367...",MWPGILVGGARVASCRYPALGPRLAAHFPAQRPPQRTLQNGLALQR...,"[0.19444445, 0.21296297, 0.23148148, 1.0, 0.46..."
1,2,"{HP:0000006, HP:0000005, HP:0000001}","{GO:0045088, GO:0009892, GO:0007596, GO:003555...","{GO:0045088, GO:0009892, GO:0007596, GO:003555...","[GO:0000003|0.189, GO:0000165|0.051, GO:000018...",MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,"[0.015174977, 0.014245897, 0.03995045, 0.00061..."
2,8195,"{HP:0012758, HP:0000107, HP:0009142, HP:000011...","{GO:0006457, GO:0071705, GO:0098840, GO:012003...","{GO:0006457, GO:0098840, GO:0045776, GO:012003...","[GO:0000003|0.595, GO:0000226|0.861, GO:000110...",MSRLEAKKPSLCKSEPLTTERVRTTLSVLKRIVTSCYGPSGRLKQL...,"[0.5, 0.71428573, 0.39285713, 0.4642857, 0.428..."
3,8200,"{HP:0009334, HP:0009179, HP:0009381, HP:001024...","{GO:0051252, GO:0030545, GO:0032774, GO:000635...","{GO:0032270, GO:0051252, GO:0009893, GO:003513...","[GO:0000003|0.143, GO:0000122|0.069, GO:000016...",MRLPKLLTFLLWYLAWLDLEFICTVLGAPDLGQRPQGTRPGLAKAE...,"[0.0125, 0.025, 0.025, 0.025, 0.025, 0.0125, 0..."
4,90121,"{HP:0009118, HP:0000119, HP:0011675, HP:000432...",{},"{GO:0008150, GO:0009987, GO:0006364, GO:002261...","[GO:0000462|0.5, GO:0006139|0.545, GO:0006364|...",MAGAAEDARALFRAGVCAALEAWPALQIAVENGFGGVHSQEKAKWL...,"[0.62211984, 0.76958525, 1.0, 0.41013825, 0.36..."
...,...,...,...,...,...,...,...
3928,147409,"{HP:0012759, HP:0100753, HP:0030669, HP:003203...","{GO:0016043, GO:0003674, GO:0030216, GO:000815...","{GO:0051252, GO:0030216, GO:0032774, GO:000635...","[GO:0000003|0.046, GO:0000165|0.01, GO:0000226...",MDWLFFRNICLLIILMVVMEVNSEFIVEVKEFDIENGTTKWQTVRR...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3929,8148,"{HP:0003324, HP:0030236, HP:0002493, HP:000209...","{GO:0009892, GO:0006367, GO:0051252, GO:003555...","{GO:0003730, GO:0009892, GO:0006367, GO:005125...","[GO:0000018|0.062, GO:0000375|0.321, GO:000037...",MSDSGSYGQSGGEQQSYSTYGNPGSQGYGQASQSYSGYGQTTDSSY...,"[0.2644628, 0.32231405, 0.446281, 1.0, 0.44628..."
3930,344018,"{HP:0000008, HP:0000001, HP:0000812, HP:000000...","{GO:0044464, GO:0003674, GO:0008134, GO:000551...","{GO:0051252, GO:0044427, GO:0031981, GO:000989...","[GO:0000122|0.059, GO:0000228|0.256, GO:000078...",MDPAPGVLDPRAAPPALLGTPQAEVLEDVLREQFGPLPQLAAVCRL...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3931,81887,"{HP:0001956, HP:0009118, HP:0012758, HP:000246...","{GO:0043227, GO:0008150, GO:0043231, GO:003198...","{GO:0003674, GO:0043227, GO:0008150, GO:004323...","[GO:0005575|0.973, GO:0005622|0.973, GO:000562...",MSWESGAGPGLGSQGMDLVWSAWYGKCVKGKGSLPLSAHGIVVAWL...,"[0.18965517, 0.23275863, 0.29310346, 0.3620689..."
